In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import scipy.sparse as sparse
import random
# import implicit
# from implicit.als import AlternatingLeastSquares as ALS

%cd /home/user_3/medistream-recsys/Script
from preprocessing import drop_columns,dict_to_column,dict_to_set,set_to_column,key_to_element

pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 100)

import warnings
warnings.filterwarnings("ignore")

/home/user_3/medistream-recsys/Script


In [2]:
df1 = pd.read_json('/fastcampus-data/select_column_version_3.json')
df = df1.copy()

In [3]:
start_date = "2022-05-12"

# paid orders만 가져오기
df['date_paid'] = pd.to_datetime(df['date_paid'])
df_only_paid = df[~df['date_paid'].isna()]

# 3개월치 데이터만 가져오기
df_date = df_only_paid[df_only_paid['date_paid'] > start_date]

# 취소 안된 것만 가져오기
complete_df = df_date[(df_date['paid'] == True) & (df_date['cancelled']==False)]
complete_df = complete_df.reset_index(drop=True)

# 도서 카테고리만 가져오기
df = complete_df[complete_df['name'] == '도서']
df = df.sort_values(by='date_paid')

In [4]:
df_group = df.groupby('customer_id')['name_x'].unique().reset_index()
df_group

,customer_id,name_x
0,5d60cab24e77525ec5ca13d5,"[흔히보는 정형외과 외래진료 가이드북, 숲을 보는 요통치료]"
1,5d60cb754e77525ec5ca13d9,[흔히보는 정형외과 외래진료 가이드북]
2,5d60cc294e77525ec5ca13e0,"[약침의 정석 –통증편, 영어 진료 가이드북 ]"
3,5d60ccd84e77525ec5ca13f2,[증보운곡본초학]
4,5d60cce84e77525ec5ca13f3,[보험한약 브런치 the # 2판 개정판]
...,...,...
2658,63155514c640eb0021af84fc,[임상 한의사를 위한 기본 한약처방 강의 2판]
2659,63155585c640eb0021af8506,[영어 진료 가이드북 ]
2660,6315b68a6c2e37002369ccd0,[한방처방정석]
2661,63195d5e6c2e37002369dd30,[윤상훈·권병조의 알짜 근육학]


In [7]:
cus_df=df.groupby(['customer_id', 'date_paid'])[['product_ids','name_x']].apply(sum)
# cus_df = cus_df.to_frame()

In [8]:
cus_df

product_ids  \
customer_id              date_paid                                                    
5d60cab24e77525ec5ca13d5 2022-05-25 23:35:50.527000+00:00  6182113bbc63410023473754   
                         2022-06-30 02:17:10.503000+00:00  617f9228bc63410023472aa4   
5d60cb754e77525ec5ca13d9 2022-05-25 15:45:24.832000+00:00  6182113bbc63410023473754   
5d60cc294e77525ec5ca13e0 2022-06-30 00:41:38.247000+00:00  628ecd6b9d9388002406f05b   
                         2022-09-03 05:24:26.386000+00:00  63046e9358e1680033a580d6   
...                                                                             ...   
63155514c640eb0021af84fc 2022-09-05 01:53:01+00:00         610b6f9ad26af90681b1e699   
63155585c640eb0021af8506 2022-09-05 04:13:53.041000+00:00  63046e9358e1680033a580d6   
6315b68a6c2e37002369ccd0 2022-09-07 10:36:06+00:00         5e93e5d6bfe426094478533e   
63195d5e6c2e37002369dd30 2022-09-08 03:17:32+00:00         60d018b7d26af90681b18980   
63199ec06c2e37002369df0f 2022-09-08 10:44:29.018000+00:00  60d018b7d26af90681b18980   

                                                                             name_x  
customer_id              date_paid                                                   
5d60cab24e77525ec5ca13d5 2022-05-25 23:35:50.527000+00:00       흔히보는 정형외과 외래진료 가이드북  
                         2022-06-30 02:17:10.503000+00:00                숲을 보는 요통치료  
5d60cb754e77525ec5ca13d9 2022-05-25 15:45:24.832000+00:00       흔히보는 정형외과 외래진료 가이드북  
5d60cc294e77525ec5ca13e0 2022-06-30 00:41:38.247000+00:00               약침의 정석 –통증편  
                         2022-09-03 05:24:26.386000+00:00               영어 진료 가이드북   
...                                                                             ...  
63155514c640eb0021af84fc 2022-09-05 01:53:01+00:00         임상 한의사를 위한 기본 한약처방 강의 2판  
63155585c640eb0021af8506 2022-09-05 04:13:53.041000+00:00               영어 진료 가이드북   
6315b68a6c2e37002369ccd0 2022-09-07 10:36:06+00:00                           한방처방정석  
63195d5e6c2e37002369dd30 2022-09-08 03:17:32+00:00                  윤상훈·권병조의 알짜 근육학  
63199ec06c2e37002369df0f 2022-09-08 10:44:29.018000+00:00           윤상훈·권병조의 알짜 근육학  

[4528 rows x 2 columns]

In [ ]:
cus_df[cus_df['customer_id'] == "5d60cab24e77525ec5ca13d5"]

In [ ]:
cus_df['customer_id']

In [20]:
cus_df.reset_index(level=['date_paid'], inplace = True)

In [31]:
print(len(cus_df.loc['5dad02370dabe405b156ee41']))
cus_df.loc['5dad02370dabe405b156ee41']

15


,date_paid,product_ids,name_x
customer_id,,,
5dad02370dabe405b156ee41,2022-05-19 00:38:09.647000+00:00,62392e146f766b0024667fe2,카이로프랙틱 기본테크닉론
5dad02370dabe405b156ee41,2022-05-23 09:33:56.015000+00:00,6182113bbc63410023473754,흔히보는 정형외과 외래진료 가이드북
5dad02370dabe405b156ee41,2022-05-25 23:00:39.004000+00:00,6182113bbc63410023473754,흔히보는 정형외과 외래진료 가이드북
5dad02370dabe405b156ee41,2022-05-30 08:49:07.408000+00:00,628df3509d9388002406ed08,趙紹琴(조소금) 내과학
5dad02370dabe405b156ee41,2022-06-02 00:25:30.699000+00:00,6285af619d93880024065341,안면마비 침구치료
5dad02370dabe405b156ee41,2022-06-02 00:32:19.251000+00:00,6285af5c9d9388002406533e,"숨찬 세상, 호흡기를 편하게"
5dad02370dabe405b156ee41,2022-06-06 23:16:18.446000+00:00,613eec926b61d0001afbbc67,시니어 피부과학
5dad02370dabe405b156ee41,2022-06-14 08:59:22.842000+00:00,613eec9a6b61d0001afbbc6b629860589d93880024071a...,나의 갑상선 진료황황교수의 임상의를 위한 근거기반 상한금궤 처방 매뉴얼황황교수의 개...
5dad02370dabe405b156ee41,2022-06-20 08:45:48.941000+00:00,628ecd6a9d9388002406f059628ecd689d9388002406f058,뇌의학의 첫걸음갑상선 진료 완전정복


In [24]:
# 많이 구매한 customer로 정렬한 것인데, 하루에 여러권의 책을 샀을 경우 count +1만 되기 때문에 value count한 값보다
# 실제로 구매한 책의 권수는 더 많을 수도 있다.
cus_df.index.value_counts().head(100)

5dad02370dabe405b156ee41    15
5d970b4f0dabe405b156dce4    15
5e94323fbfe426094478562d    12
5d9ef5c40dabe405b156e35d    11
5d931d770dabe405b156d63b    11
5d65cb104e77525ec5ca1703    11
5dae9a8d0dabe405b156efd0    10
6220687af967570023fa7e74    10
5d69cd4319efa30eb2913de3    10
5f6da2dd48060c52caf15718    10
5dce69512bb59605ca3cffb6     9
5dd61d3f2bb59605ca3d0504     9
5ed49629e510ee503167c4c2     9
5f9a94c0512031633432077c     9
5dc0cedf2bb59605ca3cf6a0     9
5de0fcc22bb59605ca3d0bee     9
5d96fec10dabe405b156dc4a     8
5d68633d19efa30eb2913d54     8
5f9a8b8c5120316334320719     8
5df0c0a44267e105dffef407     8
5d61ca9e4e77525ec5ca1573     8
60a311e20ccc56001bc7e4c3     8
6076b83dc1908b001a63e355     7
5d6f6fa819efa30eb2913fc7     7
5e888faebfe4260944784b91     7
5d71cd0f19efa30eb291433a     7
61c547d47560f900324c9e9f     7
5d96fd5f0dabe405b156dbf0     7
5d73699719efa30eb2914378     7
5f7d8c8748060c52caf1635e     7
5e5da336bfe4260944782c29     7
5d974f020dabe405b156dd75     7
5e5fb67c

In [37]:
# 15번 등장하는 customer. 실제 구매한 책은 18권 
print(len(df[df['customer_id'] == '6099ccf10ccc56001bc7d9ec']))
df[df['customer_id'] == '6099ccf10ccc56001bc7d9ec']

1


,_id,date_paid,customer_id,paid,name_x,category_id_y,product_ids,quantity,price,price_total,age_group,한의사 여부,사업자 여부,cancelled,name,slug
33687,62c2439f08e04900234e6192,2022-07-04 01:37:36.145000+00:00,6099ccf10ccc56001bc7d9ec,True,임상을 위한 의학입문강의,5cf8bbba0098b2225c5dfaa3,5f8412d95c12dc06f754db21,1.0,135000.0,135000.0,50-59,0.0,0.0,False,도서,book


In [67]:
df1[df1['name_x'] == '그림으로 보는 신경진찰법']

,_id,date_paid,customer_id,paid,name_x,category_id_y,product_ids,quantity,price,price_total,age_group,한의사 여부,사업자 여부,cancelled,name,slug
45317,6031c5e5527223001a209a78,2021-02-25T02:06:15.286Z,5ff3f9575ae455001b219bbb,True,그림으로 보는 신경진찰법,5cf8bbba0098b2225c5dfaa3,602cf65c2c593a001aceac90,1.0,49200.0,49200.0,None,1.0,1.0,False,도서,book
46304,6039fa76643a8a0019f2b439,2021-02-27T08:25:38.186Z,5f7d8c8748060c52caf1635e,False,그림으로 보는 신경진찰법,5cf8bbba0098b2225c5dfaa3,602cf65c2c593a001aceac90,1.0,49200.0,49200.0,30-39,1.0,0.0,True,도서,book
47996,604c49f5c1908b001a63acf0,2021-03-13T10:25:04.379Z,5da111c00dabe405b156e799,True,그림으로 보는 신경진찰법,5cf8bbba0098b2225c5dfaa3,602cf65c2c593a001aceac90,1.0,49200.0,49200.0,20-29,1.0,0.0,False,도서,book
70517,60a6b7430ccc56001bc7e93e,2021-05-20T20:12:15.142Z,5d6f28db19efa30eb2913f79,True,그림으로 보는 신경진찰법,5cf8bbba0098b2225c5dfaa3,602cf65c2c593a001aceac90,1.0,49200.0,49200.0,30-39,1.0,0.0,False,도서,book
74796,60cbec7fd26af90681b1862e,2021-06-18T00:52:04.975Z,5ea669221a3ca36b96784a24,True,그림으로 보는 신경진찰법,5cf8bbba0098b2225c5dfaa3,602cf65c2c593a001aceac90,1.0,49200.0,49200.0,30-39,1.0,1.0,False,도서,book
80773,60efd3f2d26af90681b1c51e,None,5d968f610dabe405b156d869,False,그림으로 보는 신경진찰법,5cf8bbba0098b2225c5dfaa3,602cf65c2c593a001aceac90,1.0,49500.0,49500.0,20-29,0.0,0.0,False,도서,book
89398,612c222f3b4626001a054d07,None,60a6bb9e0ccc56001bc7e949,False,그림으로 보는 신경진찰법,5cf8bbba0098b2225c5dfaa3,602cf65c2c593a001aceac90,1.0,49500.0,49500.0,None,1.0,1.0,False,도서,book
94897,614f580a4cf9a5002287b949,2021-09-25T17:51:12.100Z,5dae9a8d0dabe405b156efd0,True,그림으로 보는 신경진찰법,5cf8bbba0098b2225c5dfaa3,602cf65c2c593a001aceac90,1.0,49200.0,49200.0,30-39,1.0,1.0,False,도서,book
105072,617feb94bc63410023472dfa,None,616f6ea44cf9a50022880bb0,False,그림으로 보는 신경진찰법,5cf8bbba0098b2225c5dfaa3,602cf65c2c593a001aceac90,1.0,49500.0,49500.0,None,1.0,0.0,False,도서,book
111598,6198b32ebc634100234788ff,None,5eec4df2e510ee503167d006,False,그림으로 보는 신경진찰법,5cf8bbba0098b2225c5dfaa3,602cf65c2c593a001aceac90,1.0,49500.0,49500.0,20-29,1.0,0.0,False,도서,book


In [71]:
df[df['name_x'].str.contains('해부학')].customer_id

157      5fae14fc5120316334323588
256      62690f7cdc78e80024f12d6f
385      6052a3550ccc56001bc78f23
746      5de1b5ae2bb59605ca3d0c0b
18063    5dc0cedf2bb59605ca3cf6a0
18806    5db8d16a0dabe405b156f84f
19108    5d8357363f0e6805c470682b
19106    5d8357363f0e6805c470682b
19536    5d96a7050dabe405b156daf8
21328    6022444c2c593a001ace9f02
25841    5e997e27bfe4260944785d49
28576    5d931d770dabe405b156d63b
28565    5f3e3ccfa0b91001bf48d3bf
28563    5f3e3ccfa0b91001bf48d3bf
29781    6220687af967570023fa7e74
30005    5d68633d19efa30eb2913d54
30117    5effeb9a221c8b5f77c8d8bc
33000    5def2fba4267e105dffef313
33284    5dfacfb74267e105dffefb82
33398    5d9696120dabe405b156d9fc
33400    5d9696120dabe405b156d9fc
33655    5e9dae7ebfe4260944785f38
34044    5d9a8bc90dabe405b156dfdf
34381    5de0fcc22bb59605ca3d0bee
35318    60c7e60c0ccc56001bc80855
35501    5e7877cdbfe4260944783f1a
36443    603f330240a8110019dda3b6
41241    6076dac90ccc56001bc7b4e1
44033    61714ccc4cf9a500228816ec
44034    61714

In [89]:

df1[df1['customer_id'] == '5e9a8ccdbfe4260944785d8f']

,_id,date_paid,customer_id,paid,name_x,category_id_y,product_ids,quantity,price,price_total,age_group,한의사 여부,사업자 여부,cancelled,name,slug
13250,5e9a8d11bfe4260944785d92,None,5e9a8ccdbfe4260944785d8f,False,None,None,None,NaN,NaN,NaN,20-29,1.0,0.0,False,None,None
49212,6058c2a20ccc56001bc7960a,2021-03-22T16:18:08.417Z,5e9a8ccdbfe4260944785d8f,True,병태생리 Visual map,5cf8bbba0098b2225c5dfaa3,5d77b73619efa30eb29143ca,1.0,87000.0,87000.0,20-29,1.0,0.0,False,도서,book
57270,60a20b68c1908b001a6422f3,2021-05-17T06:21:28.281Z,5e9a8ccdbfe4260944785d8f,True,[제31기 전졸협] 국시특강 간계내과학,5ea40b671a3ca36b967849ad,609e4740c1908b001a641f11,1.0,0.0,0.0,20-29,1.0,0.0,False,온라인 강의,lecture
57781,60a20d10c1908b001a6426f3,2021-05-17T06:28:32.928Z,5e9a8ccdbfe4260944785d8f,True,[제31기 전졸협] 국시특강 심계내과학,5ea40b671a3ca36b967849ad,609e47410ccc56001bc7dfbb,1.0,0.0,0.0,20-29,1.0,0.0,False,온라인 강의,lecture
58291,60a20ec8c1908b001a642af5,2021-05-17T06:35:52.835Z,5e9a8ccdbfe4260944785d8f,True,[제31기 전졸협] 국시특강 비계내과학,5ea40b671a3ca36b967849ad,609e4743c1908b001a641f12,1.0,0.0,0.0,20-29,1.0,0.0,False,온라인 강의,lecture
58803,60a20f51c1908b001a642ef6,2021-05-17T06:38:09.675Z,5e9a8ccdbfe4260944785d8f,True,[제31기 전졸협] 국시특강 폐계내과학,5ea40b671a3ca36b967849ad,609e4745c1908b001a641f13,1.0,0.0,0.0,20-29,1.0,0.0,False,온라인 강의,lecture
59314,60a2102bc1908b001a6432fc,2021-05-17T06:41:48.015Z,5e9a8ccdbfe4260944785d8f,True,[제31기 전졸협] 국시특강 신계내과학,5ea40b671a3ca36b967849ad,609e4745c1908b001a641f14,1.0,0.0,0.0,20-29,1.0,0.0,False,온라인 강의,lecture
59826,60a211c0c1908b001a643700,2021-05-17T06:48:32.846Z,5e9a8ccdbfe4260944785d8f,True,[제31기 전졸협] 국시특강 상한론,5ea40b671a3ca36b967849ad,609e47460ccc56001bc7dfbc,1.0,0.0,0.0,20-29,1.0,0.0,False,온라인 강의,lecture
60340,60a213cfc1908b001a643b04,2021-05-17T06:57:19.709Z,5e9a8ccdbfe4260944785d8f,True,[제31기 전졸협] 국시특강 사상의학,5ea40b671a3ca36b967849ad,609e47470ccc56001bc7dfbd,1.0,0.0,0.0,20-29,1.0,0.0,False,온라인 강의,lecture
60849,60a21515c1908b001a643f04,2021-05-17T07:02:45.787Z,5e9a8ccdbfe4260944785d8f,True,[제31기 전졸협] 국시특강 침구학,5ea40b671a3ca36b967849ad,609e4747c1908b001a641f15,1.0,0.0,0.0,20-29,1.0,0.0,False,온라인 강의,lecture


In [62]:
type(df1.name_x.iloc[0])

NoneType

In [44]:
print(len(df['name_x'].unique()))
df['name_x'].unique()

252


array(['장골의 PI 변위는 없다', '초음파 가이드 근골격계 통증 치료의 정석', '섭혜민 명의경방험안',
       '환자상담의 달인', 'KCD 한방내과 진찰진단 가이드라인',
       '그림과 사진으로 보는 질환별 통증치료 Essential', '숲을 보는 요통치료', '트리거포인트 침치료',
       '침의 과학적 접근의 이해', '윤상훈·권병조의 알짜 근육학', '실전한약가이드', '등중갑 방제학강의',
       '말초신경 약침의학', '근골격계 약침의학', '근막시스템의 기능해부학', '핸즈온 허리',
       '척추통증_움직임 문제 (Back Pain: A Movement Problem_A clinical approach incorporating relevant research and practice)',
       '카이로프랙틱 기본테크닉론', '기본통증진료학', 'Cyriax 정형의학 3판',
       "Medical acupuncture '침의 과학적 접근과 임상활용' ",
       '임상 한의사를 위한 기본 한약처방 강의 2판', '감별진단의 정석', '토털매선의학', '소아피부질환해설',
       '알아두면 쓸모있는 비만치료', 'Netters 근육뼈대계 간결해부학', '열패혈증염증',
       '유진홍 교수의 이야기로 풀어보는 감염학', '항생제 열전', '개원의를 위한 통증사냥법', '요가해부학',
       '유경원 온병학강의', '한방 123처방 임상 해설', '일차진료 한의사를 위한 보험한약입문 - 둘째 판',
       '보험한약 브런치 the # 2판 개정판', '피부질환의 한의학치료 57증례', 'SMART 일차진료매뉴얼',
       '무릎통증(아프니까 무릎이다)', '통증백해(痛症百解)', '한의 피부진료 첫 걸음', '미안침', '고장 난 뇌',
       '임상의를 위한 멘탈한방 입문', '통증치료를 위한 알기 쉬운 근골격계 이학적 검사법',
       '통증치료를 위한